<a href="https://colab.research.google.com/github/Ameer-sayyad/ML-DL-Projects/blob/main/RNN_Using_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#RNN using pytorch

In [29]:
import pandas as pd
df = pd.read_csv("/content/100_Unique_QA_Dataset.csv")
df.head()

,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100


In [30]:
def tokenize(text):
  text = text.lower()
  text = text.replace('?','')
  text = text.replace("'","")
  return text.split()

In [31]:
tokenize('what is the capital of france?')

['what', 'is', 'the', 'capital', 'of', 'france']

In [32]:
vocab = {'<UNK>':0}

In [33]:
#Build vocab
def build_vocab(row):
  tokenized_question = tokenize(row['question'])
  tokenized_answer = tokenize(row['answer'])
  merged_tokens = tokenized_question + tokenized_answer
  for token in merged_tokens:
    if token not in vocab:
      vocab[token] = len(vocab)

In [34]:
df.apply(build_vocab,axis=1)

,0
0,None
1,None
2,None
3,None
4,None
...,...
85,None
86,None
87,None
88,None


In [35]:
len(vocab)

324

In [36]:
#convert words to numerical indices
def text_to_indices(text,vocab):
  indexed_text = []
  for token in tokenize(text):
    if token in vocab:
      indexed_text.append(vocab[token])
    else:
      indexed_text.append(vocab['<UNK>'])
  return indexed_text

In [37]:
text_to_indices("what is campus x",vocab)

[1, 2, 0, 0]

In [38]:
# Creating Q&A Dataset class
import torch
from torch.utils.data import Dataset,DataLoader
class QADataset(Dataset):
  def __init__(self,df,vocab):
    self.df = df
    self.vocab = vocab
  def __len__(self):
    return self.df.shape[0]
  def __getitem__(self, index):
    row = self.df.iloc[index]
    numerical_question = text_to_indices(row['question'],self.vocab)
    numerical_answer = text_to_indices(row['answer'],self.vocab)
    return torch.tensor(numerical_question),torch.tensor(numerical_answer)


In [39]:
dataset = QADataset(df,vocab)
dataloader = DataLoader(dataset,batch_size=1,shuffle=True)

In [40]:
for question,answer in dataloader:
  print(question,answer)

tensor([[ 10, 308,   3, 309, 310]]) tensor([[311]])
tensor([[  1,   2,   3,   4,   5, 113]]) tensor([[114]])
tensor([[ 42, 117, 118,   3, 119,  94, 120]]) tensor([[121]])
tensor([[ 42, 101,   2,   3,  17]]) tensor([[102]])
tensor([[ 1,  2,  3, 59, 25,  5, 26, 19, 60]]) tensor([[61]])
tensor([[ 42, 318,   2,  62,  63,   3, 319,   5, 320]]) tensor([[321]])
tensor([[  1,  87, 229, 230, 231, 232]]) tensor([[233]])
tensor([[ 10,  29, 130, 131]]) tensor([[132]])
tensor([[ 42,   2,   3, 210, 137, 168, 211, 169]]) tensor([[113]])
tensor([[ 42, 312,   2, 313,  62,  63,   3, 314, 315]]) tensor([[316]])
tensor([[  1,   2,   3, 163, 164, 165,  83,  84]]) tensor([[166]])
tensor([[  1,   2,   3, 122, 123,  19,   3,  45]]) tensor([[124]])
tensor([[10, 11, 12, 13, 14, 15]]) tensor([[16]])
tensor([[ 42, 167,   2,   3,  17, 168, 169]]) tensor([[170]])
tensor([[ 10, 140,   3, 141, 171,   5,   3,  70, 172]]) tensor([[173]])
tensor([[ 42, 290, 291, 118, 292, 158, 293, 294]]) tensor([[295]])
tensor([[ 1,  2

In [41]:
#Creating a Simple RNN
import torch.nn as nn
class SimpleRNN(nn.Module):

  def __init__(self,vocab_size):
    super().__init__()
    self.embedding=nn.Embedding(vocab_size,embedding_dim=50)
    self.rnn = nn.RNN(50,64,batch_first=True)
    self.fc = nn.Linear(64,vocab_size)
  def forward(self,question):
    embedded_question = self.embedding(question)
    hidden,final = self.rnn(embedded_question)
    output = self.fc(final.squeeze(0))
    return output

In [42]:
learning_rate = 0.001
epochs = 20

In [43]:
model = SimpleRNN(len(vocab))
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

In [44]:
#training loop
for epoch in range(epochs):
  total_loss = 0
  for question,answer in dataloader:
    optimizer.zero_grad()
    output = model(question)
    loss = criterion(output,answer[0])
    loss.backward()
    optimizer.step()
    total_loss += loss.item()
  print(f"Epoch: {epoch+1}, Loss: {total_loss/len(dataloader)}")

Epoch: 1, Loss: 5.804360421498616
Epoch: 2, Loss: 5.0844055334726965
Epoch: 3, Loss: 4.247327915827433
Epoch: 4, Loss: 3.5303606536653307
Epoch: 5, Loss: 2.943064166439904
Epoch: 6, Loss: 2.411453738477495
Epoch: 7, Loss: 1.9282935559749603
Epoch: 8, Loss: 1.4993847890032663
Epoch: 9, Loss: 1.1592323074738184
Epoch: 10, Loss: 0.882555236087905
Epoch: 11, Loss: 0.6876956014169587
Epoch: 12, Loss: 0.5360995989706782
Epoch: 13, Loss: 0.42885950869984096
Epoch: 14, Loss: 0.3464598793950346
Epoch: 15, Loss: 0.2866817217734125
Epoch: 16, Loss: 0.2370616663661268
Epoch: 17, Loss: 0.2018383568359746
Epoch: 18, Loss: 0.1723296651409732
Epoch: 19, Loss: 0.147205250006583
Epoch: 20, Loss: 0.12657665717932914


In [53]:
def predict(model,question,threshold=0.5):
  numerical_question = text_to_indices(question,vocab)
  question_tensor = torch.tensor(numerical_question).unsqueeze(0)
  output = model(question_tensor)
  probabilities = torch.nn.functional.softmax(output,dim=1)
  value,index = torch.max(probabilities,dim=1)
  print(value,index)
  if value<threshold:
   print("i dont know")
  print(list(vocab.keys())[index])


In [54]:
predict(model,"what is the largest planet in our solar system")


tensor([0.8894], grad_fn=<MaxBackward0>) tensor([23])
jupiter
